# Create People Collect 2

In [1]:
import pandas as pd

In [2]:
pc = pd.read_csv('people_collect/people_collect_2.csv')
direct_1790_matched = pd.read_csv('matches/direct_matches.csv')
baptisms = pd.read_csv('data/Baptisms.csv')
census_1790 = pd.read_csv('data/1790 Census Data Complete.csv')
census_1790['ecpp_id'] = range(1, len(census_1790) + 1)

## Insert Matched 

In [3]:
census_1790.columns

Index(['Current_Location', 'First', 'First_Normalized', 'Last',
       'Last_Normalized', 'Race', 'Rank', 'Gender', 'Origin', 'Age',
       'Spouse_First', 'Spouse_Last', 'Father_First', 'Father_Last',
       'Mother_First', 'Mother_Last', 'Child1', 'Child2', 'Child3', 'Child4',
       'Child5', 'Child6', 'Child7', 'Child8', 'Child9', 'Child10', 'Child11',
       'Child12', 'Child13', 'Child14', 'Origin Parish', 'Race in Other',
       'Location Other Race', 'Notes/URL', 'ecpp_id'],
      dtype='object')

In [4]:
baptisms.columns

Index(['#ID', 'DMission', 'MMission', 'FMission', 'Mission', 'DeathLinkBasis',
       'BurialDate', 'DeathNumber', 'SCOfficiant', 'Recorder', 'Officiant',
       'MNumber', 'MReligiousStatus', 'MEthnicity', 'MOrigin', 'MSurname',
       'MNativeName', 'MSpanishName', 'FMilitaryStatus', 'FReligiousStatus',
       'FEthnicity', 'FOrigin', 'FSurname', 'FNativeName', 'FSpanishName',
       'FNumber', 'BirthDate', 'DerivedOrigin', 'ReligiousStatus', 'Ethnicity',
       'Origin', 'Surname', 'NativeName', 'SpanishName', 'Age', 'Place',
       'TypePhrase', 'SupliCeremoniaDate', 'Number', 'Date', 'FmtdDate',
       'Type', 'MaritalStatus', 'Sex', 'Leg', 'Unit', 'Level', 'Notes'],
      dtype='object')

In [5]:
census_1790.Race

0               español
1               español
2              española
3       orphan, unknown
4               español
             ...       
1003          español, 
1004          español, 
1005          español, 
1006            español
1007           española
Name: Race, Length: 1008, dtype: object

In [6]:
direct_1790_matched

,ecpp_id,#ID,Census First Name,Baptisms First Name,Census Last Name,Baptisms Last Name,Is Direct Match
0,14,10949,José María,Joseph Maria,Verdugo,Verdugo,True
1,15,1603,María,Martina,López,Lopez,True
2,15,11438,María,Martina,López,[Lopez],True
3,15,11804,María,Maria,López,Lopez,True
4,15,28491,María,Mariana,López,Gomez,True
...,...,...,...,...,...,...,...
128,962,73563,José Miguel,Jose Manuel,Saez,Sayenz,True
129,975,1525,Juan,Juan,López,Jones,True
130,975,25658,Juan,Juan,López,Lopez,True
131,995,95394,Ana María,Juana Maria,Bojórquez,Bojorquez,True


In [7]:
pc.columns

Index(['last_name', 'first_name', 'race_aggregated', 'race_1790',
       'race_sj1778', 'race_la1781', 'race_la1785', 'race_la1821', 'ethnicity',
       'baptismal_date', 'location_ecpp_baptism', 'location_1790_census',
       'father_last_name', 'father_first_name', 'father_military_status',
       'father_origin', 'mother_last_name', 'mother_first_name',
       'mother_origin', 'sex', 'origin_parish_1790_census',
       'location_other_race', 'notes_url_1790_census'],
      dtype='object')

In [8]:
def clean_and_reformat_race(race_value):
    if pd.isnull(race_value):
        return []
    cleaned_list = [
        item.strip().strip('.').replace('[', '').replace(']', '')
        for item in str(race_value).split(',') if item.strip()
    ]
    return cleaned_list

## Baptisms & 1790 Census

In [9]:
# access values that are matched between baptisms & 1790 census and insert into people_collect 2.0
# will need function to get parents names
# function to aggregate race
data_to_append = []

for index, row in direct_1790_matched.iterrows():
    pc.loc[index, 'last_name'] = census_1790.loc[row['ecpp_id'], 'Last']
    pc.loc[index, 'first_name'] = census_1790.loc[row['ecpp_id'], 'First']
    pc.loc[index, 'father_last_name'] = census_1790.loc[row['ecpp_id'], 'Father_Last']
    pc.loc[index, 'father_first_name'] = census_1790.loc[row['ecpp_id'], 'Father_First']
    pc.loc[index, 'mother_last_name'] = census_1790.loc[row['ecpp_id'], 'Mother_Last']
    pc.loc[index, 'mother_first_name'] = census_1790.loc[row['ecpp_id'], 'Mother_First']
    # maybe redundant?
    pc.loc[index, 'father_origin'] = baptisms.loc[row['#ID'], 'FOrigin']
    pc.loc[index, 'mother_origin'] = baptisms.loc[row['#ID'], 'FOrigin']

    pc.loc[index, 'father_military_status'] = baptisms.loc[row['#ID'], 'FMilitaryStatus']
    pc.loc[index, 'sex'] = baptisms.loc[row['#ID'], 'Sex']

    pc.loc[index, 'race_1790'] = census_1790.loc[row['ecpp_id'], 'Race']
    pc.loc[index, 'ethnicity'] = baptisms.loc[row['#ID'], 'Ethnicity']
    pc.loc[index, 'baptismal_date'] = baptisms.loc[row['#ID'], 'FmtdDate']
    pc.loc[index, 'location_ecpp_baptism'] = baptisms.loc[row['#ID'], 'DMission']
    pc.loc[index, 'location_1790_census'] = census_1790.loc[row['ecpp_id'], 'Current_Location']
    pc.loc[index, 'origin_parish_1790_census'] = census_1790.loc[row['ecpp_id'], 'Origin Parish']
    pc.loc[index, 'location_other_race'] = census_1790.loc[row['ecpp_id'], 'Location Other Race']

    # popoulate pc_id
    data_to_append.append({'pc_id': index, '1790_ecpp_id': row['ecpp_id'], 'baptisms_id': row['#ID']})

    

pc['race_1790'] = pc['race_1790'].apply(clean_and_reformat_race)
pc['ethnicity'] = pc['ethnicity'].apply(clean_and_reformat_race)
pc_id = pd.DataFrame(data_to_append)

## Baptisms & 1821 Padron

In [10]:
# values are repetitive, but ecpp_id and baptisms id are unique. This means records are duplicated
pc.to_csv('people_collect/people_collect_2.csv', index=False)

In [11]:
pc_id.to_csv('people_collect/people_collect_2_ids.csv', index=False)

In [12]:
pc_id

,pc_id,1790_ecpp_id,baptisms_id
0,0,14,10949
1,1,15,1603
2,2,15,11438
3,3,15,11804
4,4,15,28491
...,...,...,...
128,128,962,73563
129,129,975,1525
130,130,975,25658
131,131,995,95394
